In [1]:
import scrapy
import pandas as pd

In [2]:
import requests

cookies = {
    'wordpress_sec_6ef7f7293198b3c0f850c97eb92e606b': 'tuankiet%7C1638967546%7CPwMLwJja7RweqHSmQXdBplEnsJNgO721r4YA2LH8Meu%7Ce9514307d32b0b78691384b3eb4d470c2117483677c4e55745bcb7581e688fe8',
    '_ga': 'GA1.1.2138357637.1634886347',
    'accepted-cookie-policy': 'accepted',
    'PHPSESSID': 'fp57gb9rak181o8lik5iii3stq',
    'wp-wpml_current_language': 'en',
    'wordpress_test_cookie': 'WP%20Cookie%20check',
    '_ga_TZ0PQKEQSM': 'GS1.1.1638794731.49.1.1638794743.0',
    'wordpress_logged_in_6ef7f7293198b3c0f850c97eb92e606b': 'tuankiet%7C1638967546%7CPwMLwJja7RweqHSmQXdBplEnsJNgO721r4YA2LH8Meu%7C20735243d8b83d9ba0e87e3d87e676a7cbfdd7c2dd3e60b1b67761641ee51b2e',
    'wp-wpml_current_admin_language_d41d8cd98f00b204e9800998ecf8427e': 'vi',
    'wp-settings-time-187': '1638795277',
    'AWSALB': 'O+Yj+ANnxnveW+jF4YjaHbjBgdTIrheElxnPHY5vqbXhlXi8NH7Pllnw9uIHVpokEFn3mrfkX+YlhT89i0dAW7etkFXIE75A7fPXxxXRjyqOKDWsoXaT63rSJvS+',
    'AWSALBCORS': 'O+Yj+ANnxnveW+jF4YjaHbjBgdTIrheElxnPHY5vqbXhlXi8NH7Pllnw9uIHVpokEFn3mrfkX+YlhT89i0dAW7etkFXIE75A7fPXxxXRjyqOKDWsoXaT63rSJvS+',
}

headers = {
    'authority': 'ementorhub.com',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'referer': 'https://ementorhub.com/wp-login.php?redirect_to=https%3A%2F%2Fementorhub.com%2Fwp-admin%2Fusers.php&reauth=1',
    'accept-language': 'en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7',
}


In [3]:
def cleanSelector(obj, xpath = './text()'):
    clean_list = []
    for i in range(len(obj)):
        if obj[i].xpath(xpath).extract():
            clean_list.append(obj[i].xpath(xpath).extract()[0])
        else:
            clean_list.append(None)
    return clean_list

In [4]:
links = ['https://ementorhub.com/wp-admin/users.php',
         'https://ementorhub.com/wp-admin/users.php?paged=2',
         'https://ementorhub.com/wp-admin/users.php?paged=3',
         'https://ementorhub.com/wp-admin/users.php?paged=4',
         'https://ementorhub.com/wp-admin/users.php?paged=5']

In [5]:
data_dict = {'first name':[],
             'last name': [],
             'user': [],
            'mail': [],
            'link': [],
            'role': [],
            'In 3-5 years': [],
            'The reason': [],
            'Who am I': []}
count = 0

In [6]:

for link in links:
    response = requests.get(link, headers=headers, cookies=cookies).content
    sel = scrapy.Selector(text = response)
    users = cleanSelector(sel.xpath('//*[@id="the-list"]//strong/a'))
    mails = cleanSelector(sel.xpath('//*[@id="the-list"]//td[@class = "email column-email"]/a'))
    links = cleanSelector(sel.xpath('//*[@id="the-list"]//td[@class = "username column-username has-row-actions column-primary"]/strong/a'), './@href')
    roles = cleanSelector(sel.xpath('//*[@id="the-list"]//td[@class = "role column-role"]'))
    for i in range(len(users)):
        data_dict['user'].append(users[i])
        data_dict['mail'].append(mails[i])
        data_dict['link'].append(links[i])
        data_dict['role'].append(roles[i])
        if data_dict['role'][count] == "Administrator":
            data_dict['first name'].append(None)
            data_dict['last name'].append(None)
            data_dict['In 3-5 years'].append(None)
            data_dict['The reason'].append(None)
            data_dict['Who am I'].append(None)
            print("Admin")
        elif data_dict['role'][count] == "Editor":
            data_dict['first name'].append(None)
            data_dict['last name'].append(None)
            data_dict['In 3-5 years'].append(None)
            data_dict['The reason'].append(None)
            data_dict['Who am I'].append(None)
            print("Edit")
        else:
            res_detail = requests.get(links[i], headers=headers, cookies=cookies).content
            sel_detail = scrapy.Selector(text = res_detail)
            info = cleanSelector(sel_detail.xpath('//*[@id="your-profile"]/table[7]/tbody/tr[29]//input'), './@value')
            reason = cleanSelector(sel_detail.xpath('//*[@id="your-profile"]/table[7]/tbody/tr[30]//input'), './@value')
            selfdef = cleanSelector(sel_detail.xpath('//*[@id="your-profile"]/table[7]/tbody/tr[13]//input'), './@value')
            firstname = cleanSelector(sel_detail.xpath('//*[@id="first_name"]'), './@value')
            lastname = cleanSelector(sel_detail.xpath('//*[@id="last_name"]'), './@value')
            print(lastname, firstname, info, reason, selfdef, "member", users[i])
            data_dict['In 3-5 years'].append(info[0])
            data_dict['The reason'].append(reason[0])
            data_dict['Who am I'].append(selfdef[0])
            data_dict['first name'].append(firstname[0])
            data_dict['last name'].append(lastname[0])
        count += 1

df = pd.DataFrame(data_dict)

df.to_excel('CustomerDataSummary.xlsx', index = True)
display(df)
# print(data_dict)

[None] [None] [None] member 18120256@student.hcmus.edu.vn
[None] [None] [None] member ai17802.ai@gmail.com
[None] [None] ['14 năm trong lĩnh vực kinh doanh và marketing trong các tập đoàn quốc tế, hiện là tư vấn tài chính của ngân hàng Hoàng Gia Canada'] member alanvo1510@gmail.com
[None] [None] [None] member anguy012@uottawa.ca
[None] [None] [None] member anh.nguyen@acs-aec.ca
Admin
[None] [None] [None] member baogia01022021@gmail.com
['coordinated a pilot project with Environics to improve data informed decision making at the City of Ottawa'] ['I have a chance to work with many talented people in the field of data science.'] ['The nerd amongst the cool kids'] member bendick2@gmail.com
[None] [None] [None] member bichtuyen84@gmail.com
[None] [None] [None] member buiphuong362@gmail.com
['kỹ năng phân tích chuyên nghiệp, thành thạo công cụ phân tích'] ['Mong muốn theo đuổi ngành Data Analyst'] ['Là một người hòa đồng, thân thiện như nhiều bạn bè nhận xét. \nThích du lịch  để khám phá mi

[None] [None] ['Mình là một người đam mê du lịch, thích trải nghiệm và học hỏi những kiến thức mới. Bên cạnh đó cũng là người thích chia sẻ, hướng dẫn và đồng hành cho các bạn. Sinh viên.'] member phamhong.vmg@gmail.com
[None] ['#'] ['#'] member phamtam.tn@gmail.com
['Trong 3-5 năm tới, tôi muốn chuyển công việc của tôi từ một nhà nghiên cứu sang một nhà phân tích kinh doanh.'] ['Như tôi biết, các nhà phân tích kinh doanh thường được trả lương cao hơn các nhà nghiên cứu.'] ['Dinh Phan'] member phanhuudinh@hotmail.com
[None] [None] [None] member phongnguyen.learning@gmail.com
[None] [None] [None] member phungnguyen1594@gmail.com
[None] [None] [None] member phuonganh.2x2g@gmail.com
Edit
['Tôi lead 1 team 65 members ở 4 quốc gia phát triển hệ thống loyalty và logistics được coi là business backbone systems cho phía đối tác.'] ['Tôi gặp và làm việc với nhiều bạn executive ở các công ty đa quốc gia để học hỏi thêm và cũng gặp với nhiều bạn freshers và giúp đỡ về định hướng cho các bạn trong

,first name,last name,user,mail,link,role,In 3-5 years,The reason,Who am I
0,,,18120256@student.hcmus.edu.vn,18120256@student.hcmus.edu.vn,https://ementorhub.com/wp-admin/user-edit.php?...,Mentee,None,None,None
1,,,ai17802.ai@gmail.com,ai17802.ai@gmail.com,https://ementorhub.com/wp-admin/user-edit.php?...,Mentee,None,None,None
2,Alan,Vo,alanvo1510@gmail.com,alanvo1510@gmail.com,https://ementorhub.com/wp-admin/user-edit.php?...,Mentor,None,None,14 năm trong lĩnh vực kinh doanh và marketing ...
3,,,anguy012@uottawa.ca,anguy012@uottawa.ca,https://ementorhub.com/wp-admin/user-edit.php?...,Mentee,None,None,None
4,,,anh.nguyen@acs-aec.ca,anh.nguyen@acs-aec.ca,https://ementorhub.com/wp-admin/user-edit.php?...,Mentor,None,None,None
...,...,...,...,...,...,...,...,...,...
89,,,tuyetnhungspring@gmail.com,tuyetnhungspring@gmail.com,https://ementorhub.com/wp-admin/user-edit.php?...,Mentor,None,None,None
90,Quang Vũ,Đàm,vu.q.dam@gmail.com,vu.q.dam@gmail.com,https://ementorhub.com/wp-admin/user-edit.php?...,Mentor,đã làm việc với những vị trí liên quan đến Dat...,tôi học được rất nhiều về Data và gặp được rất...,Em là 1 người hứng thú với những trải nghiệm m...
91,,,wearethesame@gmail.com,None,https://ementorhub.com/wp-admin/user-edit.php?...,Mentee,None,None,None
92,Lê,Thọ,wearethesame13@gmail.com,wearethesame13@gmail.com,https://ementorhub.com/wp-admin/user-edit.php?...,Mentee,None,None,haha
